# COGS 108 - Data Checkpoint

# Names

- Zihan Chen
- Ning Ma
- Kancheng Yin
- Nan Jiang

<a id='research_question'></a>
# Research Question

**General question**: What makes an airplane crash more fatal? 


**Specific question**: 
1. Does the fatality rate associate with a specific time of the day and specific location of the world? 
2. Does the fatality rate associate with who's operataing the plane *(private vs. military personale)*?
3. Is there any keywords are more likely to associate with high fatality accident *(e.g., "birds", "air traffic", "operator errors")*?



# Dataset(s)

- **Dataset Name:** Airplane Crashes and Fatalities From 1908 to 2009
- **Link to the dataset:** https://data.world/data-society/airplane-crashes
- **Number of observations:** 5268 x 13
- **Description of the dataset:** This dataset contains the list of airplane crashes and fatalities happened from 1908 to 2009, and it also includes all information regrarding certain flight(['Date', 'Time', 'Location', 'Operator', 'Flight #', 'Route', 'Type',
       'Registration', 'cn/In', 'Aboard', 'Fatalities', 'Ground', 'Summary'])

<br>

- **Dataset Name:** Supplementary Aviation Accidents in Countries Since 1919
- **Link to the dataset:** https://www.kaggle.com/datasets/ramjasmaurya/aviation-accidents-history1919-april-2022
- **Number of observations:** 21699 x 8 
- **Description of the dataset:** This dataset contains the list of airplane crashes and fatalities happened since 1919, and it also includes all information regrarding certain flight(['Country', 'date', 'Air-craft type', 'registration name/mark',
       'operator', 'fatilites', 'location', 'category'])


We will mainly use first dataset since we have more information about certain airplane crash which we can analyze with. The problem with the first dataset is that it is not up-to-date, so we will use second dataset to supplement data from 2009 to 2022.

# Setup

### Import Needed Libraries

In [1]:
# Import numpy to handle numerical information and analyse data
import numpy as np

# Import pandas to read csv file and to work with dataframes
import pandas as pd

# Import matplotlib and seaborn to visulize data
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
sns.set_context('talk')

# Import math to perform mathematical operations
import math

# Import patsy and statsmodels to perform regression
import patsy
import statsmodels.api as sm

# Import scipy to perform statistical operations
import scipy.stats as stats

# Set warnings to ignore
import warnings
warnings.filterwarnings('ignore')

### Import Data

In [2]:
# Load the airplane crashes and fatalities data from 1908 to 2009
# We get this dataset from data.world(https://data.world/data-society/airplane-crashes)
df_main = pd.read_csv('data/Airplane_Crashes_and_Fatalities_From_1908_to_2009.csv')

# Load the supplimentary aviation accidents data from 1919 to 2022 which 
# we will only use data from 2009 to 2022
# We get this dataset from Kaggle(https://www.kaggle.com/datasets/ramjasmaurya/aviation-accidents-history1919-april-2022)
df_supplementary = pd.read_csv('data/supplementary_aviation_accidents.csv')

# Data Cleaning

We have two data, and we decide to clean them seperately and then merge.

### First, we clean the airplane crash data

As mentioned before, 

**df_main** has columns ['Date', 'Time', 'Location', 'Operator', 'Flight #', 'Route', 'Type',
       'Registration', 'cn/In', 'Aboard', 'Fatalities', 'Ground', 'Summary'] 
       
**df_supplementary** has columns ['Country', 'date', 'Air-craft type', 'registration name/mark',
       'operator', 'fatilites', 'location', 'category']

With our research problem, we only need ['Date', 'Location', 'Operator', 'Route', 'Type', 'Aboard', 'Fatalities', 'Summary'] columns. We will extract columns ['Country', 'date', 'Air-craft type', 'operator', 'fatilites', 'location'] from supplementary data and convert them so that two datasets have same format in same type of data. 

In [3]:
# This function is used to unify the date format to be year/month/day
def unify(data):
    data = str(data).split('/')
    if len(data) < 3:
        return ''
    return data[2] + '/' + data[0] + '/' + data[1]

# Remain the data that required by our research problem
columns_final = ['Date', 'Time', 'Location', 'Operator', 'Route', 'Type', 'Aboard', 'Fatalities', 'Summary']
df_main = df_main[columns_final]

# unify the date format
df_main['Date'] = df_main['Date'].apply(unify)


Now we can take a look at the dataset

In [4]:
df_main.head()

,Date,Time,Location,Operator,Route,Type,Aboard,Fatalities,Summary
0,1908/09/17,17:18,"Fort Myer, Virginia",Military - U.S. Army,Demonstration,Wright Flyer III,2.0,1.0,"During a demonstration flight, a U.S. Army fly..."
1,1912/07/12,06:30,"AtlantiCity, New Jersey",Military - U.S. Navy,Test flight,Dirigible,5.0,5.0,First U.S. dirigible Akron exploded just offsh...
2,1913/08/06,NaN,"Victoria, British Columbia, Canada",Private,NaN,Curtiss seaplane,1.0,1.0,The first fatal airplane accident in Canada oc...
3,1913/09/09,18:30,Over the North Sea,Military - German Navy,NaN,Zeppelin L-1 (airship),20.0,14.0,The airship flew into a thunderstorm and encou...
4,1913/10/17,10:30,"Near Johannisthal, Germany",Military - German Navy,NaN,Zeppelin L-2 (airship),30.0,30.0,Hydrogen gas which was being vented was sucked...


Then we will transform supplementary data to the format that main dataset requires

In [5]:
import calendar
# Notice that month in this dataset is in string format, we need to convert it to number
month_to_number = {month: str(index) for index, month in enumerate(calendar.month_abbr) if month}
for month in month_to_number:
    if len(month_to_number[month]) == 1:
        month_to_number[month] = "0" + month_to_number[month]

# This function is used to unify the date format to be year/month/day for this dataset
def filter_date(data):
    if isinstance(data, float):
        return ''
    
    data = str(data).split('-')
    if len(data) < 3:
        return ''
    if data[1] in month_to_number:
        data[1] = month_to_number[data[1]]
    
    if data[1] == '???': 
        return '/'.join([data[2], '01/01'])
    elif data[0] == '??': 
        return '/'.join([data[2], data[1], '01'])
    elif len(data) == 3:
        return '/'.join([data[2], data[1], data[0]])
    else:
        return ''

# Notice that there are weird input for fatalities(e.g. 8+1)
# So this function is used to filter the fatalities data
def filter_fatalities(data):
    data = str(data)
    if "+" in data:
        return sum([int(x) for x in data.split('+')])
    else:
        return int(data) if data.isdigit() else 0

# This function is clean the data to the format that we need in order to append to main dataframe
def transform(df_supplementary):
    df_filtered = df_supplementary[['date', 'Country', 'location', 'operator', 'Air-craft type', 'fatilites']]

    # unify the name for the columns
    df_filtered = df_filtered.rename(columns={'date': 'Date', 'location': 'Location', 'operator': 'Operator', 
                                                'Air-craft type': 'Type', 'fatilites': 'Fatalities'})

    # append the country name to the location
    df_filtered['Location'] = df_filtered['Location'] + ", " + df_filtered['Country']

    # Replace "near" in the location because we will draw heatmap with this data
    df_filtered['Location'] = df_filtered['Location'].apply(lambda x: str(x).replace('near', ''))

    # filter the date with designed functions
    df_filtered['Date'] = df_filtered['Date'].apply(filter_date)
    df_filtered['Fatalities'] = df_filtered['Fatalities'].apply(filter_fatalities)
    
    return df_filtered.drop(columns=['Country'])

df_supplementary = transform(df_supplementary)


Now we can take a look at the processed supplementary dataset

In [6]:
df_supplementary.head()

,Date,Location,Operator,Type,Fatalities
0,2016/12/07,"Hobart Airpo..., Australia",Virgin Australia Airlines,Boeing 737-8FE (WL),0
1,1987/02/19,"Stadnitsa, Ukraine",Soviet AF,Antonov An-26,9
2,2003/02/19,"Kerman, Iran",Iranian Revolutionary Guard,Ilyushin Il-76MD,275
3,1990/05/11,"Rodenbach, Germany",German AF,Transall C-160D,10
4,1943/01/04,"Ostrow, Russia",German AF,Junkers Ju-52/3m,0


Since We just need data from 2009 to 2022, we will append only them to the main dataframe

In [7]:
# We will only use data from 2009 to 2022
df_supplementary = df_supplementary[pd.to_datetime(df_supplementary['Date']) > "01-Jan-2009"]

# Since there are three columns this dataframe does not have, so we need to add them
df_supplementary[['Route', 'Summary']] = " "
df_supplementary['Aboard'] = -1
df_supplementary['Time'] = np.nan

# Reorder the columns
df_supplementary = df_supplementary[['Date', 'Time', 'Location', 'Operator', 'Route', 'Type', 'Aboard', 'Fatalities', 'Summary']]

## Second, we merge the data

Now we do the last cleaning which is removing airplane crash that does not have date andreplacing invalid value to default value

In [8]:
df = df_main.append(df_supplementary).reset_index(drop=True)

# free memory
del df_main
del df_supplementary

# We replace the empty data to NaN and drop rows with NaN date
for column in columns_final:
    df[column] = df[column].replace('', np.nan)
df.dropna(subset=['Date'])
df[['Aboard', 'Fatalities']] = df[['Aboard', 'Fatalities']].fillna(0)
# Change these two columns to int
df['Aboard'] = df['Aboard'].astype(int)
df['Fatalities'] = df['Fatalities'].astype(int)

# We save this dataframe to csv file for future use
df.to_csv('data/df.csv')

## The Final DataFrame
- **Date:** The year/month/day of airplane crash
- **Location:** Location of the airplane crash
- **Operator:** The institute/company/government/office that operate the flight
- **Route:** The Route of the flight
- **Type:** Airplane type of the flight
- **Aboard:** Aboard number of the flight
- **Fatalities:** Fatalities of the airplane crash
- **Summary:** Brief summary for what causes this airplane crash

In [9]:
df.head()

,Date,Time,Location,Operator,Route,Type,Aboard,Fatalities,Summary
0,1908/09/17,17:18,"Fort Myer, Virginia",Military - U.S. Army,Demonstration,Wright Flyer III,2,1,"During a demonstration flight, a U.S. Army fly..."
1,1912/07/12,06:30,"AtlantiCity, New Jersey",Military - U.S. Navy,Test flight,Dirigible,5,5,First U.S. dirigible Akron exploded just offsh...
2,1913/08/06,NaN,"Victoria, British Columbia, Canada",Private,NaN,Curtiss seaplane,1,1,The first fatal airplane accident in Canada oc...
3,1913/09/09,18:30,Over the North Sea,Military - German Navy,NaN,Zeppelin L-1 (airship),20,14,The airship flew into a thunderstorm and encou...
4,1913/10/17,10:30,"Near Johannisthal, Germany",Military - German Navy,NaN,Zeppelin L-2 (airship),30,30,Hydrogen gas which was being vented was sucked...
